In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:

dataset_path = "data/yield_df.csv"

# Reading the CSV file
df = pd.read_csv(dataset_path)


In [ ]:
df.head()

# Feature Selection

a. Domain Knowledge


1.   **pesticides(TU)**: Pesticide usage might directly affect crop yield.
2.   **Land_per_1000ha**: Land area could influence yield.
3.   **average_rain_fall_mm_per_year**: Rainfall is critical for crop growth.
4.   **avg_temp**: Temperature affects crop growth and yield.
5.   **Year**: Time trends might influence yield due to technological advancements or climate change.





  
  

  

b. Correlation Analysis

In [ ]:
# Creating a pairplot of the data
sns.pairplot(data=df,hue='Item',kind='scatter');

In [ ]:

#Removing Categorical Data
df.drop(columns=['Country','Item','Code','Indicator Name','Indicator Code'], inplace=True) # Dropping unnecessary index column
# Checking for missing values and drop them
df.dropna(inplace=True)
corr_matrix = df.corr() # Correlation matrix
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()

**Correlation Analysis**
1. Most Important Features for kg/ha_yield:
  

  *   Item Code (FAO) (0.16): Moderate positive correlation.
  *   Year (0.089): Weak positive correlation.
  *   pesticides(TU) (0.075): Weak positive correlation.

2. Multicollinearity Check:
  
  *   pesticides(TU) and Land_per_1000ha are highly correlated (0.59). Including both in the model might introduce multicollinearity. We have to remove one of them.

3. Weak or Irrelevant Features:


  *   average_rain_fall_mm_per_year: Very weak correlation with all features, including yield. Likely not useful for the model.

  *   avg_temp: Weak negative correlation with yield.

***But according to domain knowledge , average_rain_fall_mm_per_year and avg_temp , these two features may be dependent on yield. ***











c. Multicollinearity Check

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Creating a subset of the data with the continous independent variables.
X = df

# Calculating the variance inflation factor for each variable.
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Creating a DataFrame with the VIF results for the column names in X.
df_vif = pd.DataFrame(vif, index=X.columns, columns = ['VIF'])

# Adding a new column for interpretation
df_vif['Interpretation'] = df_vif['VIF'].apply(lambda x:
    'No multicollinearity' if x < 1 else
    'Low multicollinearity' if x < 5 else
    'Moderate multicollinearity' if x < 10 else
    'Severe multicollinearity'
)
# Displaying the VIF results.
df_vif

According to VIF report values, Year and avg_temp have severe multicollinearity. So , these two features should be considered for removal from the model.

3. Check Features One by One

In [ ]:
import statsmodels.api as sm

# Simple linear regression for each feature
for feature in df.columns:
    if feature != "kg/ha_yield":  # Skipping the target variable
        X = sm.add_constant(df[feature])  # Adding a constant for the intercept
        model = sm.OLS(df["kg/ha_yield"], X).fit()
        print(f"Feature: {feature}")
        print(model.summary())
        print("\n***********************************\n")

Regression Summary for All Features (Dependent Variable: kg/ha_yield)

1.   Area Code (M49)

  *   Effect: Weak negative (-1.57 kg/ha per unit increase)
  *  Significance: Highly significant (p < 0.0001)
  *  R²: 0.002 (Very low predictive power)


2.   Year

  *  Effect: Positive (85.84 kg/ha increase per year)
  *  Significance: Highly significant (p < 0.0001)
  *  R²: 0.008 (Still low predictive power)
  *  Concern: High condition number (potential multicollinearity)

3. Item Code (FAO)

  *  Effect: Positive (12.96 kg/ha per unit increase)
  *  Significance: Highly significant (p < 0.0001)
  *  R²: 0.026 (Higher but still low)
  *  Concern: Non-normal residuals

4. Pesticides (TU)

  *  Effect: Very small positive (0.01 kg/ha per unit increase)
  *  Significance: Highly significant (p < 0.0001)
  *  R²: 0.006 (Minimal impact on yield)
  *  Concern: High condition number (potential multicollinearity)

5. Land per 1000 ha

  *  Effect: Small positive (0.36 kg/ha per unit increase)
  *  Significance: Highly significant (p < 0.0001)
  *  R²: 0.004 (Very low predictive power)

6. Average Rainfall (mm/year)

  *  Effect: Weak negative (-2.89 kg/ha per mm increase)
  *  Significance: Not statistically significant (p = 0.069)
  *  R²: ~0 (No meaningful relationship)

7. Average Temperature (°C)

  *  Effect: Strong negative (-119.5 kg/ha per °C increase)
  *  Significance: Highly significant (p < 0.0001)
  *  R²: 0.012 (Still weak predictor but better than most)


## **Building the Multiple Regression Model**

In [ ]:
df.rename(columns={'Area Code (M49)': 'AreaCode', 'Item Code (FAO)': 'ItemCode',
                  'pesticides(TU)': 'Pesticides','kg/ha_yield':'kg_pr_ha_yield'}, inplace=True)


In [ ]:
df.info()

In [ ]:
from statsmodels.formula.api import ols
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Splitting the data into training (80%) and testing (20%) sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Defining the OLS formula
ols_formula_1 = 'kg_pr_ha_yield ~ AreaCode + ItemCode + Pesticides + average_rain_fall_mm_per_year + avg_temp'

# Fitting the model using only the training set
OLS_1 = ols(formula=ols_formula_1, data=train_df)
model_1 = OLS_1.fit()

# Making predictions on the test set
y_pred = model_1.predict(test_df)

# Calculating evaluation metrics on the test set
r2 = r2_score(test_df['kg_pr_ha_yield'], y_pred)
mse = mean_squared_error(test_df['kg_pr_ha_yield'], y_pred)
mae = mean_absolute_error(test_df['kg_pr_ha_yield'], y_pred)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Printing model summary (trained on training set)
print(model_1.summary())

# Printing performance metrics on test data
print("\nModel Performance Metrics (on Test Set):")
print(f"R-squared (Test): {r2:.4f}")
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")



In [ ]:
# Calculating the residuals.

# Extracting actual values from train and test sets
y_train = train_df['kg_pr_ha_yield']
y_test = test_df['kg_pr_ha_yield']


# Computing residuals for test data
residuals = y_test - y_pred

plt.figure(figsize=(14, 6))

# Predicted vs Actual values plot
plt.subplot(1, 2, 1)
plt.scatter(y_test, y_pred, alpha=0.5)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Predicted vs Actual')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)  # 45-degree reference line

# Residuals plot
plt.subplot(1, 2, 2)
plt.scatter(y_pred, residuals, alpha=0.5)
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residuals Plot')
plt.axhline(y=0, color='r', linestyle='--', lw=2)  # Horizontal line at zero

plt.show()



Predicted vs. Actual Plot (Left)

The x-axis represents actual values, and the y-axis represents predicted values.
The red dashed line represents the ideal case where predictions match actual values.
The points appear clustered near the bottom, indicating that the model tends to underestimate higher values.
Residuals Plot (Right)

The x-axis represents predicted values, and the y-axis represents residuals (errors).
The red dashed line at zero indicates an ideal error distribution.
The spread of residuals increases with predicted values, suggesting potential heteroscedasticity (variance of errors increasing with larger values), which might indicate model issues.


# Add additional features to improve predictions

In [ ]:
df['rainfall_temp'] = df['average_rain_fall_mm_per_year'] * df['avg_temp']
df['pesticide_land'] = df['Pesticides'] * df['Land_per_1000ha']


In [ ]:
from statsmodels.formula.api import ols
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Splitting the data into training (80%) and testing (20%) sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Defining the OLS formula
ols_formula_2 = 'kg_pr_ha_yield ~ AreaCode + ItemCode + Pesticides + average_rain_fall_mm_per_year + avg_temp + rainfall_temp +pesticide_land'

# Fitting the model using only the training set
OLS_2 = ols(formula=ols_formula_2, data=train_df)
model_2 = OLS_2.fit()

# Making predictions on the test set
y_pred = model_2.predict(test_df)

# Calculating evaluation metrics on the test set
r2 = r2_score(test_df['kg_pr_ha_yield'], y_pred)
mse = mean_squared_error(test_df['kg_pr_ha_yield'], y_pred)
mae = mean_absolute_error(test_df['kg_pr_ha_yield'], y_pred)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Printing model summary (trained on training set)
print(model_2.summary())

# Printing performance metrics on test data
print("\nModel Performance Metrics (on Test Set):")
print(f"R-squared (Test): {r2:.4f}")
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")


In [ ]:

# Calculating the residuals.

# Extracting actual values from train and test sets
y_train = train_df['kg_pr_ha_yield']
y_test = test_df['kg_pr_ha_yield']


# Computing residuals for test data
residuals = y_test - y_pred

plt.figure(figsize=(14, 6))

# Predicted vs Actual values plot
plt.subplot(1, 2, 1)
plt.scatter(y_test, y_pred, alpha=0.5)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Predicted vs Actual')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)  # 45-degree reference line

# Residuals plot
plt.subplot(1, 2, 2)
plt.scatter(y_pred, residuals, alpha=0.5)
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residuals Plot')
plt.axhline(y=0, color='r', linestyle='--', lw=2)  # Horizontal line at zero

plt.show()

# Show the plot.
plt.show()

# Conclusion:

*   The second model didn't show significant improvement.

*   While MAE slightly improved, R² dropped and RMSE increased, suggesting the new features aren't adding value.

*   A Random Forest model could better capture complex relationships, handle feature interactions, and improve robustness, potentially leading to better performance.

# Apply Random Forest


In [ ]:
df.info()

In [ ]:
# Splitting data into training and testing blocks
X = df.drop(['kg_pr_ha_yield'], axis=1)
y = df['kg_pr_ha_yield']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.50, random_state=42)

X_test.head()


In [ ]:
y_test.head()

In [ ]:
y_train.head()

In [ ]:
X_train.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=2)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)  # Root Mean Squared Error


# Printing performance metrics on test data
print("\nModel Performance Metrics (on Test Set):")
print(f"R-squared (Test): {r2:.4f}")
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")

In [ ]:

# Computing residuals for test data
residuals = y_test - y_pred

plt.figure(figsize=(14, 6))

# Predicted vs Actual values plot
plt.subplot(1, 2, 1)
plt.scatter(y_test, y_pred, alpha=0.5)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Predicted vs Actual')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)  # 45-degree reference line

# Residuals plot
plt.subplot(1, 2, 2)
plt.scatter(y_pred, residuals, alpha=0.5)
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residuals Plot')
plt.axhline(y=0, color='r', linestyle='--', lw=2)  # Horizontal line at zero

plt.show()



The first graph illustrates that the model performs well, despite some variations from the optimal line. Indicators of potential heteroscedasticity are present in the residuals plot, which displays a funnel shape, suggesting that the model might have overlooked some data.



1. **R-squared (R²):**
    *   Random Forest: 0.9294 (strong predictive power)
    *   Multiple Linear Regression: 0.0564 (very weak predictive power)

2. **Mean Squared Error (MSE):**
    *   Random Forest: 4,537,729.41 (much lower error)
    *   Multiple Linear Regression: 60,936,594.21 (significantly higher error)

3. **Mean Absolute Error (MAE):**
    *   Random Forest: 872.26
    *   Multiple Linear Regression: 5708.26

4. **Root Mean Squared Error (RMSE):**
    *   Random Forest: 2130.19
    *   Multiple Linear Regression: 7806.19

# Conclusion
The Random Forest model outperforms Multiple Linear Regression significantly. 
It has a much higher R² value (indicating better fit), and considerably lower MSE, MAE, and RMSE values (indicating lower error). 
This suggests that Random Forest is a far better choice for this prediction task.


